# SETUP PROJECT 1

Importeer pandas voor DataFrames en pyodbc voor database-connecties:

In [59]:
import sqlite3
import pandas as pd
import pyodbc
import re


aenc_conn = sqlite3.connect("../../../Data/Raw/Sqlite/aenc.sqlite")
adventureworks_conn = sqlite3.connect("../../../Data/Raw/Sqlite/AdventureWorks.sqlite")

Database Connectie:

In [60]:
DB_SDM = {"servername": r"143.177.235.181", "database": "SDM_DEDS_P1", "username": "sa", "password": "Str0ngP@ssword"}
DB_NorthWind = {"servername": r"143.177.235.181", "database": "NorthWind", "username": "sa", "password": "Str0ngP@ssword"}
export_conn = pyodbc.connect(
    f"DRIVER={{ODBC Driver 18 for SQL Server}};"
    f"SERVER={DB_SDM['servername']};"
    f"DATABASE={DB_SDM['database']};"
    f"UID={DB_SDM['username']};"
    f"PWD={DB_SDM['password']};"
    f"Encrypt=yes;TrustServerCertificate=yes;"
)

NorthWind_conn = pyodbc.connect(
    f"DRIVER={{ODBC Driver 18 for SQL Server}};"
    f"SERVER={DB_NorthWind['servername']};"
    f"DATABASE={DB_NorthWind['database']};"
    f"UID={DB_NorthWind['username']};"
    f"PWD={DB_NorthWind['password']};"
    f"Encrypt=yes;TrustServerCertificate=yes;"
)

DataFrames maken voor tables:

In [61]:
def create_dataframes_sql(connection, db_type):
    dictionary : dict = {}
    query : str = ""
    key : str = ""

    if db_type == "sqlite":
        query = "SELECT name FROM sqlite_master WHERE type='table';"
        key = "name"
    elif db_type == "ssms":
        query = "SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE';"
        key = "TABLE_NAME"

    table_names = pd.read_sql(query, connection)

    for table in table_names[key].tolist():
        try:
            dictionary[table] = pd.read_sql(f"SELECT * FROM {table}", connection)
        except Exception as e:
            print(f"Error reading {table}: {e}")
            continue

    return dictionary


aenc = create_dataframes_sql(aenc_conn, "sqlite")
adventureworks = create_dataframes_sql(adventureworks_conn, "sqlite")
northwind = create_dataframes_sql(NorthWind_conn, "ssms")

#Errors en warnings zijn niet erg hier als het totaal 1 error en 4 UserWarnings zijn.

Error reading sysdiagrams: Could not decode to UTF-8 column 'definition' with text '������'


C:\Users\saadz\AppData\Local\Temp\ipykernel_16872\1853655394.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  table_names = pd.read_sql(query, connection)
C:\Users\saadz\AppData\Local\Temp\ipykernel_16872\1853655394.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dictionary[table] = pd.read_sql(f"SELECT * FROM {table}", connection)
C:\Users\saadz\AppData\Local\Temp\ipykernel_16872\1853655394.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dictionary[table] = pd.read_sql(f"SELECT * FROM {table}", connection)
C:

Dictionaries met DataFrames mergen naar één Dictionary:

In [62]:
sdm = aenc | adventureworks | northwind #Alle drie mergen in één Dictionary met alle DataFrames

sdm['Department'] = sdm['Department'].rename(columns={"dept_id": "DepartmentID", "dept_name": "DepartmentName"})
sdm['HumanResources_Department'] = sdm["HumanResources_Department"].rename(columns={"Name": "DepartmentName"})
sdm['Department'] = pd.concat([sdm["Department"], sdm["HumanResources_Department"]], ignore_index=True)
del sdm['HumanResources_Department']

sdm['Employee'] = sdm['Employee'].rename(columns={'emp_id': 'EmployeeID', 
                                                  'emp_fname': 'FirstName', 
                                                  'emp_lname': 'LastName', 
                                                  'city': 'City', 
                                                  'state': 'Region', 
                                                  'zip_code': 'PostalCode', 
                                                  'phone': 'HomePhone', 
                                                  'start_date': 'HireDate', 
                                                  'birth_date': 'BirthDate'})
sdm['Employees'] = pd.concat([sdm['Employees'], sdm['Employee']], ignore_index=True)
sdm['Employees']
del sdm['Employee']

sdm['Product'] = sdm['Product'].rename(columns={'id': 'ProductID'})
sdm['Employees'] = sdm['Employees'].rename(columns={'dept_id': 'department_id'})

sdm['Category'] = sdm.pop('Categories')
sdm['Employee'] = sdm.pop('Employees')
sdm['Territory'] = sdm.pop('Territories')
sdm['Employee_Territories'] = sdm.pop('EmployeeTerritories')
sdm['Supplier'] = sdm.pop('Suppliers')
sdm['Shipper'] = sdm.pop('Shippers')
sdm['Order'] = sdm.pop('Orders')
sdm['Order_Detail'] = sdm.pop('Order_Details')
sdm['Sales_SalesOrderDetails'] = sdm.pop('Sales_SalesOrderDetail')

# alles lowercase maken omdat het anders niet overeenkomt met SDM database (erg bedankt Jarno...):

def transform_column_name(col):
    col = re.sub(r'([a-z])([A-Z])', r'\1_\2', col)  
    return col.lower()

def transform_dataframe_columns(df_dict):
    updated_dict = {}
    for table_name, df in df_dict.items():
        df = df.rename(columns=lambda col: transform_column_name(col))
        updated_dict[table_name] = df
    return updated_dict

sdm = transform_dataframe_columns(sdm)

# Namen van attributen alsnog handmatig veranderen (alweer bedankt Jarno...):

sdm['Customer'] = sdm['Customer'].rename(columns={
    'id': 'customer_id',
    'state': 'state'
})

sdm['Bonus'] = sdm['Bonus'].rename(columns={
    'emp_id': 'employee_id',
})

sdm['Sales_Order'] = sdm['Sales_Order'].rename(columns={
    'cust_id': 'customer_id',
})

sdm['Sales_Order_Item'] = sdm['Sales_Order_Item'].rename(columns={
    'prod_id': 'product_id',
})

sdm['HumanResources_Employee'] = sdm['HumanResources_Employee'].rename(columns={
    'national_idnumber': 'national_id_number'
})

sdm['Person_Address'] = sdm['Person_Address'].rename(columns={
    'state_province_id': 'state_province'
})

sdm['Production_BillOfMaterials'] = sdm['Production_BillOfMaterials'].rename(columns={
    'bomlevel': 'bom_level'
})

sdm['Production_Product'] = sdm['Production_Product'].rename(columns={
    'discontinued_date': 'discontinued'
})

sdm['Purchasing_PurchaseOrderDetail'] = sdm['Purchasing_PurchaseOrderDetail'].rename(columns={
    'received_qty': 'revieved_qty'
})

sdm['Sales_SalesOrderDetails'] = sdm['Sales_SalesOrderDetails'].rename(columns={
    'sales_order_detail_id': 'sales_order_details_id'
})

dict_order = [
    'Product',
    'Department',
    'State',
    'Employee',
    'Customer',
    'Bonus',
    'Sales_Order',
    'Sales_Order_Item',
    'Region',
    'Territory',
    'Employee_Territories',
    'Category',
    'Supplier',
    'Products',
    'Customers',
    'CustomerDemographics',
    'CustomerCustomerDemo',
    'Shipper',
    'Order',
    'Order_Detail',
    'Person_Person',
    'Person_Address',
    'HumanResources_Employee',
    'Production_BillOfMaterials',
    'Production_ProductCategory',
    'Production_Product',
    'Purchasing_PurchaseOrderHeader',
    'Purchasing_PurchaseOrderDetail',
    'Purchasing_Vendor',
    'Sales_SalesTerritory',
    'Sales_Store',
    'Sales_Customer',
    'Sales_SalesOrderHeader',
    'Sales_SalesOrderDetails'
]               #Hierin komt dictionary order te staan.
                #Dit moet omdat de volgorde van welke tables worden gestuurd naar database belangrijk is.

sdm = {k: sdm[k] for k in dict_order if k in sdm} #Verandert de volgorde van sdm-Dictionary.

SDM in SSMS vullen:

In [63]:
import pandas as pd
import pyodbc

def format_value(value, column_name, table_name):
    reserved_keywords = ['order', 'select', 'from', 'insert', 'update', 'delete', 'where', 'join', 'into', 'group', 'name']

    if column_name.lower() in reserved_keywords:
        column_name = f"[{column_name}]"

    if pd.isna(value):
        return "NULL"

    if isinstance(value, bool):
        return "1" if value else "0"

    if column_name.lower() in ['home_phone', 'postal_code']:
        return f"'{str(value)}'"
    
    elif isinstance(value, str):
        if value == "":
            return "NULL"
        return f"'{value.replace("'", "''")}'"
    
    elif isinstance(value, pd.Timestamp):
        return f"'{value.strftime('%Y-%m-%d')}'"
    
    elif isinstance(value, bytes):
        hex_value = value.hex()
        return f"0x{hex_value}"
    
    elif isinstance(value, (int, float)):
        return str(value)
    
    return f"'{str(value)}'"

def format_table_name(table_name):
    reserved_keywords = ['order', 'select', 'from', 'insert', 'update', 'delete', 'where', 'join', 'into']

    if table_name.lower() in reserved_keywords:
        return f"[{table_name}]"
    
    return table_name 

export_cursor = export_conn.cursor()

for table_name, df in sdm.items():

    error_count = 0
    total_count = 0

    try:
        print(f"Loading table {table_name}:", end=" ")

        formatted_table_name = format_table_name(table_name)

        for index, row in df.iterrows():
            total_count += 1

            columns = df.columns.tolist()

            if 'rowguid' in columns:
                columns.remove('rowguid')

            formatted_columns = [
                f"[{col}]" if col.lower() in ['order', 'select', 'from', 'insert', 'update', 'delete', 'where', 'join', 'into', 'group', 'name'] else col
                for col in columns
            ]

            values = []
            for col in columns:
                value = row[col]
                values.append(format_value(value, col, table_name))

            column_names = ", ".join(formatted_columns)
            value_string = ", ".join(values)

            query = f"INSERT INTO {formatted_table_name} ({column_names}) VALUES ({value_string})"
            try:
                export_cursor.execute(query)
            except pyodbc.Error as e:
                error_count += 1
                continue
    
        export_conn.commit()
        

        passed_count = total_count - error_count

        if error_count == 0:
            print(f"100% ✓")
        else:
            print(f"{passed_count}/{total_count} rows ✓")

    except pyodbc.Error as e:
        print("✗ ERROR")
        print(f"Error message: {e}")
        print("-" * 80)

export_cursor.close()


Loading table Product: 100% ✓
Loading table Department: 100% ✓
Loading table State: 100% ✓
Loading table Employee: 100% ✓
Loading table Customer: 100% ✓
Loading table Bonus: 100% ✓
Loading table Sales_Order: 100% ✓
Loading table Sales_Order_Item: 

KeyboardInterrupt: 